# Sets
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/sets.ipynb)

|      instance      | persistence | empty |   member  |   insert  |
|:------------------:|:-----------:|:-----:|:---------:|:---------:|
| Binary Search Tree |  ephemeral  |  O(1) |    O(n)   |    O(n)   |
|   Red-Black Tree   |  ephemeral  |  O(1) | O(log(n)) | O(log(n)) |

In [ ]:
class Set s where

    -- | Construct new (empty) set
    empty :: Ord a => s a
    
    -- | Check whether a set contains given item
    member :: Ord a => a -> s a -> Bool
    
    -- | Add new item to a set while maintaining the item uniqueness property
    insert :: Ord a => a -> s a -> s a

## Unbalanced Set
Implementation of an unbalanced set via a *Binary Search Tree (BST)*.

In [ ]:
data Tree a = Empty | Node (Tree a) a (Tree a)

instance Set Tree where

    -- | Construct an empty set in O(1).
    empty = Empty
    
    -- | Check whether this set contains given item.
    -- | Since the underlying BST may not be balanced, this function may take O(n) steps in the worst case.
    member _ Empty = False
    member x (Node a y b) = case (compare x y) of
        EQ -> True
        LT -> member x a
        GT -> member x b
    
    -- | Add new item to this set if it's not present yet.
    -- | Similarly to 'member', for an unbalanced instance this may take up to O(n) steps.
    insert x Empty = Node Empty x Empty
    insert x s @ (Node a y b) = case (compare x y) of
        EQ -> s
        LT -> Node (insert x a) y b
        GT -> Node a y (insert x b)

## Balanced Set
Implementation of a balanced set via a [Red-Black Tree](https://en.wikipedia.org/wiki/Red%E2%80%93black_tree) without any fancy optimizations. Specifically, in `ins` (e.g. for the left child) dosn't have to check for all the red-red violations in `balance` (actually it does not have to check the color of any node not on the search path).

In [ ]:
data Color = R | B

data Tree a = Empty | Node Color (Tree a) a (Tree a)

-- | Re-balance and locally repair the RBT color property by pushing
-- | one of two consecutive red nodes with a black parent up the path to the root.
balance :: Color -> Tree a -> a -> Tree a -> Tree a
balance B (Node R (Node R a x b) y c) z d = Node R (Node B a x b) y (Node B c z d)
balance B (Node R a x (Node R b y c)) z d = Node R (Node B a x b) y (Node B c z d)
balance B a x (Node R (Node R b y c) z d) = Node R (Node B a x b) y (Node B c z d)
balance B a x (Node R b y (Node R c z d)) = Node R (Node B a x b) y (Node B c z d)
balance color a x b = Node color a x b

instance Set Tree where 

    -- | Construct an empty set in O(1).
    empty = Empty
    
    -- | Check whether this set contains given item.
    -- | Since the underlying RBT is balanced, this function takes O(log(n)) steps in the worst case.
    member _ Empty = False
    member x (Node _ a y b) = case (compare x y) of
        EQ -> True
        LT -> member x a
        GT -> member x b
    
    -- | Add new item to this set if it's not present yet.
    -- |
    -- | Call to 'insert' takes at most O(log(n)) steps because the tree is kept balanced by
    -- | 'balance' when backing up after adding new node and the fact that in a RB tree the deepest
    -- | leaf is at most twice as far from the root as the shallowest leaf is.
    insert x Empty = Node R Empty x Empty
    insert x s = let (Node _ a y b) = ins s in Node B a y b
        where
            ins Empty = Node R Empty x Empty
            ins s @ (Node color a y b) = case (compare x y) of
                EQ -> s
                LT -> balance color (ins a) y b
                GT -> balance color a y (ins b)